# Video Dataset Transformation  

In this notebook, we show examples of video dataset transformation

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("../../")
import os
import time
import decord
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score
import torch
import torch.cuda as cuda
import torch.nn as nn
import torchvision
import urllib.request
import shutil

from utils_cv.action_recognition.data import show_batch, VideoDataset
from utils_cv.action_recognition.model import DEFAULT_MEAN, DEFAULT_STD
from utils_cv.action_recognition import system_info
from utils_cv.action_recognition.functional_video import denormalize
from utils_cv.action_recognition.transforms_video import (
    CenterCropVideo,    
    NormalizeVideo,
    RandomCropVideo,
    RandomHorizontalFlipVideo,
    RandomResizedCropVideo,
    ResizeVideo,
    ToTensorVideo,
)

system_info()

In [ ]:
def show_clip(clip, size_factor=600):
    """Show frames in a clip"""
    if isinstance(clip, torch.Tensor):
        # Convert [C, T, H, W] tensor to [T, H, W, C] numpy array 
        clip = np.moveaxis(clip.numpy(), 0, -1)
    
    figsize = np.array([clip[0].shape[1]*len(clip), clip[0].shape[0]]) / size_factor
    plt.tight_layout()
    fig, axs = plt.subplots(1, len(clip), figsize=figsize)
    for i, f in enumerate(clip):
        axs[i].axis("off")
        axs[i].imshow(f)

## Prepare a Sample Video
A sample video path:

In [ ]:
url = "https://cvbp.blob.core.windows.net/public/datasets/action_recognition/drinking.mp4"
VIDEO_PATH = os.path.join("../../data/drinking.mp4")
# Download the file from `url` and save it locally under `file_name`:
with urllib.request.urlopen(url) as response, open(VIDEO_PATH, 'wb') as out_file:
    shutil.copyfileobj(response, out_file)

In [ ]:
video_reader = decord.VideoReader(VIDEO_PATH)
video_length = len(video_reader)
print("Video length = {} frames".format(video_length))

We use three frames (the first, middle, and the last) to quickly visualize video transformations.

In [ ]:
clip = [
    video_reader[0].asnumpy(),
    video_reader[video_length//2].asnumpy(),
    video_reader[video_length-1].asnumpy(),
]
show_clip(clip)

In [ ]:
# [T, H, W, C] numpy array to [C, T, H, W] tensor
t_clip = ToTensorVideo()(torch.from_numpy(np.array(clip)))
t_clip.shape

## Video Transformations

Resizing with the original ratio

In [ ]:
show_clip(ResizeVideo(size=800)(t_clip))

Resizing

In [ ]:
show_clip(ResizeVideo(size=800, keep_ratio=False)(t_clip))

Center cropping

In [ ]:
show_clip(CenterCropVideo(size=800)(t_clip))

Random cropping

In [ ]:
random_crop = RandomCropVideo(size=800)
show_clip(random_crop(t_clip))

In [ ]:
show_clip(random_crop(t_clip))

Random resized cropping

In [ ]:
random_resized_crop = RandomResizedCropVideo(size=800)
show_clip(random_resized_crop(t_clip))

In [ ]:
show_clip(random_resized_crop(t_clip))

Normalizing (and denormalizing to verify)

In [ ]:
norm_t_clip = NormalizeVideo(mean=DEFAULT_MEAN, std=DEFAULT_STD)(t_clip)
show_clip(norm_t_clip)

In [ ]:
show_clip(denormalize(norm_t_clip, mean=DEFAULT_MEAN, std=DEFAULT_STD))

Horizontal flipping

In [ ]:
show_clip(RandomHorizontalFlipVideo(p=.5)(t_clip))